In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import time
from itertools import product
from tqdm import tqdm
import ast
import sqlite3
import json

from src.pt.PTOperator import PTOperator

# Variables

In [2]:
# Demand Size
demand_size_list = [i for i in range(100, 601, 50)]
# Demand Split Ratio (Intra Modal, %)
demand_split_ratio_list = [i for i in range(10, 91, 20)]

# Random Seed
random_seed_list = [3, 6, 9]

# Network Name
network_name = "11-500"

# Train Headway (min)
train_headway_list = [i for i in range(10, 31, 5)]

# Bus Headway (min)
bus_headway_list = [10]

# Street Station Transfer Time (seconds)
street_station_transfer_time = 60

# Mobility Hub IDs
MOBILITY_HUB_IDS = {120: "A", 241: "B"}

# Walking Speed (m/s)
WALKING_SPEED = 1.33

# Simulation Date
SIMULATION_DATE = datetime(2025, 10, 1)

# PT Network Name list
pt_network_name_list = ["basic", "ring", "diagonal", "mature"]

# Demand File Directory
pt_demand_dir = f"data/demand/{network_name}/pt/"

# GTFS File Directory
gtfs_filepath = f"data/GTFS/pt"

# Results Output Directory
results_output_dir = f"data/pt-sim-results/"
os.makedirs(results_output_dir, exist_ok=True)

# Network

In [3]:
# Load Network Files
network_name = "11-500"
network_path = f"data/network/{network_name}/"
node_df = pd.read_csv(network_path + "nodes.csv")
link_df = pd.read_csv(network_path + "edges.csv")

node_id_list = node_df['node_index'].tolist()
print(f"Number of nodes: {len(node_id_list)}")

left_node_id_list = [i for i in range(0, len(node_id_list)//2)]
right_node_id_list = [i for i in range(len(node_id_list)//2, len(node_id_list))]
print(f"Number of left nodes: {len(left_node_id_list)}")
print(f"Number of right nodes: {len(right_node_id_list)}")

left_mobility_node_id_list = [116, 117, 118, 119, 120]
right_mobility_node_id_list = [237, 238, 239, 240, 241]

Number of nodes: 242
Number of left nodes: 121
Number of right nodes: 121


In [4]:
# Load Distance Matrix
distance_matrix = np.load(network_path + "dist_matrix.npy")
print(f"Distance matrix shape: {distance_matrix.shape}")

Distance matrix shape: (242, 242)


# Database

In [5]:
DB_FILE = os.path.join(results_output_dir, 'pt_journey_plan_database.db')

def setup_database():
    if os.path.exists(DB_FILE):
        try:
            os.remove(DB_FILE)
            print(f"Successfully removed old database: {DB_FILE}")
        except OSError as e:
            print(f"Error: Could not remove file {DB_FILE}. (Is it locked?): {e}")
            return
        
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS trip_logs (
        unique_request_id INTEGER PRIMARY KEY,
        request_id REAL,
        pt_network_name TEXT,
        train_headway REAL,
        bus_headway REAL,
        trip_data TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute(create_table_sql)
            print(f"Database '{DB_FILE}' and table 'trip_logs' are ready.")
    except sqlite3.Error as e:
        print(f"An error occurred while setting up database: {e}")

def save_trip_to_db(details_dict, trip_dict):
    try:
        trip_json = json.dumps(trip_dict)
    except TypeError as e:
        print(f"Error serializing trip_dict to JSON: {e}")
        return

    insert_sql = """
    INSERT INTO trip_logs (
        unique_request_id,
        request_id, 
        pt_network_name, 
        train_headway, 
        bus_headway, 
        trip_data
    ) VALUES (?, ?, ?, ?, ?, ?)
    ON CONFLICT(unique_request_id) DO UPDATE SET
        request_id=excluded.request_id,
        pt_network_name=excluded.pt_network_name,
        train_headway=excluded.train_headway,
        bus_headway=excluded.bus_headway,
        trip_data=excluded.trip_data,
        created_at=CURRENT_TIMESTAMP;
    """
    
    data_tuple = (
        details_dict['unique_request_id'],
        details_dict['request_id'],
        details_dict['pt_network_name'],
        details_dict['train_headway'],
        details_dict['bus_headway'],
        trip_json
    )
    
    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute(insert_sql, data_tuple)
    except sqlite3.Error as e:
        print(f"An error occurred while inserting data: {e}")

In [6]:
setup_database()

Successfully removed old database: data/pt-sim-results/pt_journey_plan_database.db
Database 'data/pt-sim-results/pt_journey_plan_database.db' and table 'trip_logs' are ready.


# Run Simulations

In [ ]:
counter = 0
unique_rq_id = 0
all_demand_combinations = list(product(demand_size_list, demand_split_ratio_list, random_seed_list))

# For each PT network
for pt_network_name in pt_network_name_list:
    for train_headway in train_headway_list:
        for bus_headway in bus_headway_list:
            print(f"Running PT Simulations for PT Network: {pt_network_name}, Train Headway: {train_headway} min, Bus Headway: {bus_headway} min")
            gtfs_dir = f"data/GTFS/pt/{pt_network_name}_pt_train-{train_headway}_bus-{bus_headway}"
            # Initialize PT Operator
            pt_operator = PTOperator(gtfs_dir, print_logs=False)

            # Save the results
            save_dir = os.path.join(results_output_dir, f"{pt_network_name}_pt_train-{train_headway}_bus-{bus_headway}")
            os.makedirs(save_dir, exist_ok=True)

            # For each demand file
            for demand_size, demand_split_ratio, random_seed in tqdm(all_demand_combinations):
                demand_filepath = os.path.join(pt_demand_dir, pt_network_name, f"pt_ds{demand_size}_dsr{demand_split_ratio}_rs{random_seed}.csv")
                demand_df = pd.read_csv(demand_filepath)

                counter += 1

                # Add new columns for PT results
                demand_df["start_station_departure_time"] = np.nan
                demand_df["end_station_arrival_time"] = np.nan
                demand_df["end_station_street_arrival_time"] = np.nan
                demand_df["pt_journey_duration"] = np.nan
                demand_df["num_transfers"] = np.nan
                demand_df["selected_start_station_id"] = ''
                demand_df["selected_end_station_id"] = ''
                demand_df["walking_time"] = np.nan
                demand_df["walking_arrival_time"] = np.nan
                demand_df["best_option"] = 'pt'  # Default to PT

                for index, row in demand_df.iterrows():
                    source_station_ids = row["start_station_street_ids"].replace(";", ",")
                    source_station_ids = source_station_ids.replace(" ", "")
                    source_station_ids = ast.literal_eval(source_station_ids)
                    source_station_ids = [MOBILITY_HUB_IDS.get(item, item) for item in source_station_ids]

                    target_station_ids = row["end_station_street_ids"].replace(";", ",")
                    target_station_ids = target_station_ids.replace(" ", "")
                    target_station_ids = ast.literal_eval(target_station_ids)
                    target_station_ids = [MOBILITY_HUB_IDS.get(item, item) for item in target_station_ids]

                    station_street_arrival_time = row["start_station_street_arrival_time"]
                    arrival_time = station_street_arrival_time + street_station_transfer_time
                    arrival_datetime = SIMULATION_DATE + timedelta(seconds=arrival_time)

                    demand_df.at[index, "start_station_departure_time"] = arrival_time

                    pt_journey, selected_source_station_id, selected_target_station_id = pt_operator.return_fastest_pt_journey_xtox(
                        source_station_ids,
                        target_station_ids,
                        arrival_datetime,
                        max_transfers=999,
                        detailed=True
                    )

                    if pt_journey is not None:
                        duration = pt_journey["duration"]

                        demand_df.at[index, "end_station_arrival_time"] = arrival_time + duration
                        demand_df.at[index, "end_station_street_arrival_time"] = arrival_time + duration + street_station_transfer_time
                        demand_df.at[index, "pt_journey_duration"] = duration
                        demand_df.at[index, "num_transfers"] = pt_journey["num_transfers"]
                        demand_df.at[index, "selected_start_station_id"] = selected_source_station_id
                        demand_df.at[index, "selected_end_station_id"] = selected_target_station_id

                    rq_type = row["rq_type"]
                    if rq_type == 'intra':
                        walking_distance = distance_matrix[row["start"], row["end"]]
                        walking_time = int(walking_distance / WALKING_SPEED)
                        demand_df.at[index, "walking_time"] = walking_time
                        demand_df.at[index, "walking_arrival_time"] = row["rq_time"] + walking_time

                        # Compare PT and Walking options
                        pt_arrival_time = demand_df.at[index, "end_station_street_arrival_time"]
                        walking_arrival_time = demand_df.at[index, "walking_arrival_time"]

                        if walking_arrival_time < pt_arrival_time:
                           demand_df.at[index, "best_option"] = 'walking'

                    # Save each trip to the database
                    details_dict = {
                        'unique_request_id': unique_rq_id,
                        'request_id': row["request_id"],
                        'pt_network_name': "pt_network_name",
                        'train_headway': "train_headway",
                        'bus_headway':"bus_headway",
                    }
                    unique_rq_id += 1

                    save_trip_to_db(details_dict, pt_journey)

                # Save the demand DataFrame
                demand_df.to_csv(os.path.join(save_dir, f"pt_ds{demand_size}_dsr{demand_split_ratio}_rs{random_seed}.csv"), index=False)

print(f"Total PT simulations run: {counter}")
print(f"Total requests processed: {unique_rq_id}")

# Total PT simulations run: 3300
# Total requests processed: 1155000

Running PT Simulations for PT Network: basic, Train Headway: 10 min, Bus Headway: 10 min


100%|██████████| 165/165 [07:54<00:00,  2.87s/it]


Running PT Simulations for PT Network: basic, Train Headway: 15 min, Bus Headway: 10 min


100%|██████████| 165/165 [07:42<00:00,  2.80s/it]


Running PT Simulations for PT Network: basic, Train Headway: 20 min, Bus Headway: 10 min


...

Running PT Simulations for PT Network: mature, Train Headway: 25 min, Bus Headway: 10 min


100%|██████████| 165/165 [13:53<00:00,  5.05s/it]


Running PT Simulations for PT Network: mature, Train Headway: 30 min, Bus Headway: 10 min


100%|██████████| 165/165 [13:55<00:00,  5.06s/it]

Total PT simulations run: 3300
Total requests processed: 1155000
